# Fine-tuning custom embedding models on synthetic data

Bootstrapping and maintaining production-ready RAG pipelines, requires optimizaiton various components like the LLM, vector database, embeddings and rerankers. Within this notbeook we will showcase how you can optimize and maintain your embedding models through synthetic data and human feedback. Besides ZenML, we will do this by using two open source libraries: `argilla` and `distilabel`. Both of these libraries focus optimizing model outputs through improving data quality, however, each one of them take a diferent approach to tackle the same problem. `distilabel` provides a scalable and reliable approach to distilling knowledge from LLMs by generating synthetic data or providing AI feedback with LLMs as judges. `argilla` enables AI engineers and domain experts to collaborate on data projects by allowing them to organize and explore data through within an interactive and engagig UI. Both libraries can be used individually but they work better together.

- ⚗️ distilabel is a framework for synthetic data and AI feedback - [docs](distilabel.argilla.io)
- Argilla is a collaboration tool for AI engineers and domain experts to build high-quality datasets - [docs](docs.argilla.io)

## The dataset - vibe check

Before starting any project, it is always important to look at your data. Our data is publicly [available on the Hugging Face Hub](https://huggingface.co/datasets/zenml/rag_qa_embedding_questions_0_60_0) so we can have a quick look through their dataset viewer within an embedded iFrame. 

As we can see, our dataset contains a column called `page_content`, which was obtained from the ZenML docs.

TODO: add context about `page_content`, chunking etc. Could we look into semantic chunking?

<iframe src="https://huggingface.co/datasets/zenml/rag_qa_embedding_questions_0_60_0/embed/viewer" frameborder="0" width="100%" height="560px"></iframe>

Alternatively, we can load the entire dataset to disk with `datasets.load_dataset`. There is only a single split (`train`), which we will provide as argument to the function.

In [4]:
from datasets import load_dataset

repo_name = "zenml/rag_qa_embedding_questions_0_60_0"

dataset = load_dataset(repo_name, split="train")

dataset

Generating train split:   0%|          | 0/1656 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/414 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['page_content', 'parent_section', 'url', 'token_count'],
        num_rows: 1656
    })
    test: Dataset({
        features: ['page_content', 'parent_section', 'url', 'token_count'],
        num_rows: 414
    })
})

## Generate synthetic query generation with `distilabel`

The [`GenerateSentencePair`](https://distilabel.argilla.io/latest/components-gallery/tasks/generatesentencepair/) component from `distilabel` that can be used to generate training datasets for embeddings models. It is a pre-defined `Task` that given an `anchor` sentence generates a `positive` sentence related to the anchor. We will also generate unrelated `negative` sentences by passing `triplet=True` and we will also provide a `context` to guide the LLM towards more specific behavior. 

Additionally, we will use the [`OpenAILLM`](https://distilabel.argilla.io/latest/components-gallery/llms/openaillm/) with `gpt4o` and [`LoadDataFromHub`](https://distilabel.argilla.io/latest/components-gallery/steps/loaddatafromhub/) to load [our dataset](https://huggingface.co/datasets/zenml/rag_qa_embedding_questions_0_60_0).

In our case, we will use the `page_content` column from our dataset as `anchor` to generate `positive` and `negatives` sentences that function as training data for the embedding model.

Now, let's capture this logic in a `distilabel` `Pipeline`!

In [10]:
import os

from distilabel.steps.tasks import GenerateSentencePair
from distilabel.llms import OpenAILLM
from distilabel.steps import LoadDataFromHub
from distilabel.pipeline import Pipeline

# TODO: I think we might optimize this a bit more.

context = (
"""
The text is a chunk from technical documentation of ZenML.
ZenML is an MLOps + LLMOps framework that makes your infrastructure and workflow metadata accessible to data science teams.
Along with prose explanations, the text chunk may include code snippets and logs but these are identifiable from the surrounding backticks.
"""
)

llm = OpenAILLM(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))
# llm = OpenAILLM(model="gpt-4o", api_key="KEY_GOES_HERE")

with Pipeline(name="generate_embedding_queries") as pipeline:
    load_dataset = LoadDataFromHub(
        num_examples=10,  # uncomment this for demo purposes
        output_mappings={"page_content": "anchor"},
    )
    generate_sentence_pair = GenerateSentencePair(
        triplet=True,  # `False` to generate only positive
        action="query",
        llm=llm,
        input_batch_size=10,
        context=context,
    )

    load_dataset >> generate_sentence_pair

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 import os                                                                                   │
│    2                                                                                             │
│ ❱  3 from distilabel.steps.tasks import GenerateSentencePair                                     │
│    4 from distilabel.llms import OpenAILLM                                                       │
│    5 from distilabel.steps import LoadDataFromHub                                                │
│    6 from distilabel.pipeline import Pipeline                                                    │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │      artifact = ArtifactVersionResponse(                                                     │ │
│ │                 │   body=ArtifactVersionResponseBody(                                        │ │
│ │                 │   │   created=datetime.datetime(2024, 6, 26, 9, 47, 37),                   │ │
│ │                 │   │   updated=datetime.datetime(2024, 6, 26, 9, 47, 37),                   │ │
│ │                 │   │   user=UserResponse(                                                   │ │
│ │                 │   │   │   body=UserResponseBody(                                           │ │
│ │                 │   │   │   │   created=datetime.datetime(2024, 1, 11, 9, 20, 12),           │ │
│ │                 │   │   │   │   updated=datetime.datetime(2024, 6, 27, 10, 44, 22),          │ │
│ │                 │   │   │   │   active=True,                                                 │ │
│ │                 │   │   │   │   activation_token=None,                                       │ │
│ │                 │   │   │   │   full_name='Alex Strick van Linschoten',                      │ │
│ │                 │   │   │   │   email_opted_in=True,                                         │ │
│ │                 │   │   │   │   is_service_account=False,                                    │ │
│ │                 │   │   │   │   is_admin=False                                               │ │
│ │                 │   │   │   ),                                                               │ │
│ │                 │   │   │   metadata=None,                                                   │ │
│ │                 │   │   │   resources=None,                                                  │ │
│ │                 │   │   │   id=UUID('f4b344ce-95d8-43d4-a4f7-98ccfa119b43'),                 │ │
│ │                 │   │   │   permission_denied=False,                                         │ │
│ │                 │   │   │   name='alex.ext@zenml.io'                                         │ │
│ │                 │   │   ),                                                                   │ │
│ │                 │   │   artifact=ArtifactResponse(                                           │ │
│ │                 │   │   │   body=ArtifactResponseBody(                                       │ │
│ │                 │   │   │   │   created=datetime.datetime(2024, 4, 9, 14, 34, 3),            │ │
│ │                 │   │   │   │   updated=datetime.datetime(2024, 4, 9, 14, 34, 3),            │ │
│ │                 │   │   │   │   tags=[],                                                     │ │
│ │                 │   │   │   │   latest_version_name='13',                                    │ │
│ │                 │   │   │   │                                                                │ │
│ │                 latest_version_id=UUID('a755edef-5a62-46b

Next, we can execute this using `pipeline.run`. We will provide some `parameters` to specific components within our pipeline.

In [ ]:
from datasets import DatasetDict

train_distiset = pipeline.run(  #
    parameters={
        load_dataset.name: {
            "repo_id": "zenml/rag_qa_embedding_questions_0_60_0",
            "split": "train",
        },
        generate_sentence_pair.name: {
            "llm": {
                "generation_kwargs": {
                    "temperature": 0.7,
                    "max_new_tokens": 512,
                }
            }
        },
    },
    use_cache=False, # comment out for demo
)

test_distiset = pipeline.run(  #
    parameters={
        load_dataset.name: {
            "repo_id": "zenml/rag_qa_embedding_questions_0_60_0",
            "split": "test",
        },
        generate_sentence_pair.name: {
            "llm": {
                "generation_kwargs": {
                    "temperature": 0.7,
                    "max_new_tokens": 512,
                }
            }
        },
    },
    use_cache=False, # comment out for demo
)

combined_distiset = DatasetDict({
    "train": train_distiset["default"]["train"],
    "test": test_distiset["default"]["test"]
})

Let's vibe check our data again. If you are not happy with the results you can either tweak our `parameters` or optimize the `context` prompt which is passed to the LLM.

In [ ]:
from rich import print

example = train_distiset["default"]["train"][9]
print(example)

### (Optional) Push the distiset to the Hugging Face Hub

Synthetic data generation can be expensive becuae of the reliance on LLMs, so first store our data on the Hub.

In [ ]:
distiset.push_to_hub(
    repo_id="zenml/rag_qa_embedding_questions_0_60_0",
    token=os.getenv("HUGGINGFACE_API_KEY"),
    create_pr=True, # TODO: why?
)

### (Optional) Review synthetic query generation with `argilla` 

Data is never as clean as it can be and this also holds for synthetically generated data, therefore, it is always good to spent some time and look at your data. We will used Argilla to do this. If you are not familiar with Argilla, we recommend taking a look at the [Argilla quickstar docs](https://docs.argilla.io/latest/getting_started/quickstart/). Alternatively, you can use your Hugging Face account to login to the [Argilla demo Space](https://argilla-argilla-template-space.hf.space).

To start exploring data, we first need to define an `argilla.Dataset`. We will create a basic datset with some input `TextFields` for the `anchor` and output `TextQuestions` for the `positive` and `negative` pairs. Additionally, we will use the `parent_section` and `token_count` as `MetaDataProperty` and we will be adding some vectors to allow for semantic search. Finally, we will also compute current similarities for embeddings of `anchor-positive`, `positive-negative` and `anchor-negative` pairs.

In [ ]:
dataset = load_dataset("zenml/rag_qa_embedding_questions_0_60_0", split="train")

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model_id = "sentence-transformers/all-MiniLM-L6-v2"  # Hugging Face model ID

model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

In [ ]:
import argilla as rg

settings = rg.Setting(
    fields=[
        rg.TextField("anchor")
    ],
    questions=[
        rg.TextQuestion("positive"),
        rg.TextQuestion("negative")
    ],
    metadata=[
        rg.TermsMetadataProperty("parent_section"),
        rg.IntegerMetadataProperty("token_count"),
        rg.FloatMetadataProperty("similarity-positive-negative"),
        rg.FloatMetadataProperty("similarity-anchor-positive"),
        rg.FloatMetadataProperty("similarity-anchor-negative"),
    ],
    vectors=[
        rg.VectorField("anchor-vector", dimensions=model.get_sentence_embedding_dimension())
    ]
)
ds = rg.Dataset(
    name="rag_qa_embedding_questions_0_60_0",
    settings=settings
)
ds.create()

Next, we will process the original Hugging Face dataset.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def format_data(batch):
    def get_embeddings(batch_column):
        vectors = model.encode(batch_column)
        return [vector.tolist() for vector in vectors]
    batch["anchor-vector"] = get_embeddings(batch["anchor"])
    batch["positive-vector"] = get_embeddings(batch["positive"])
    batch["negative-vector"] = get_embeddings(batch["negative"])

    def get_similarities(a, b):
        similarities = []
        
        for pos_vec, neg_vec in zip(a, b):
            similarity = cosine_similarity([pos_vec], [neg_vec])[0][0]
            similarities.append(similarity)
    
    batch["similarity-positive-negative"] = get_similarities(batch["positive-vector"], batch["negative-vector"])
    batch["similarity-anchor-positive"] = get_similarities(batch["anchor-vector"], batch["positive-vector"])
    batch["similarity-anchor-negative"] = get_similarities(batch["anchor-vector"], batch["negative-vector"])
    return batch

dataset = dataset.map(format_data, batched=True, batch_size=1000)

Lastly, we will log the records to Argilla.

In [ ]:
records = []
for idx, entry in enumerate(dataset):
    records.append(
        rg.Record(
            id=idx,
            fields={"achor": entry["anchor"]},
            suggestions=[
                rg.Suggestion("positive", value=entry["positive"]),
                rg.Suggestion("negative", value=entry["negative"]),
            ],
            metadata={
                "parent_section": entry["parent_section"],
                "token_count": entry["token_count"],
                "similarity-positive-negative": entry["similarity-positive-negative"],
                "similarity-anchor-positive": entry["similarity-anchor-positive"],
                "similarity-anchor-negative": entry["similarity-anchor-negative"]
            },
            vectors={"question-vector": entry["question-vector"]}
        )
    )
ds.records.log(records)

Now we can explore the UI and filter out the bad apples. Tip, start filtering on high similarity of 'similarity-anchor-negative' or 'similarity-positive-negative' and low similarity of 'similarity-anchor-positive'.

# Prepare the embedding dataset

Follows [Phil Schmid's tutorial](https://www.philschmid.de/fine-tune-embedding-model-for-rag#5-evaluate-fine-tuned-model-against-baseline) fairly heavily.

In [ ]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("zenml/rag_qa_embedding_questions_0_60_0", split="train")

# Add an id column to the dataset
dataset = dataset.add_column("id", range(len(dataset)))

# split dataset into a 10% test set
dataset = dataset.train_test_split(test_size=0.1)

train_dataset_path = "../data/train_dataset.json"
test_dataset_path = "../data/test_dataset.json"
# save datasets to disk
dataset["train"].to_json(train_dataset_path, orient="records")
dataset["test"].to_json(test_dataset_path, orient="records")

# Create baseline + evaluate pretrained model

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets

model_id = "sentence-transformers/all-MiniLM-L6-v2"  # Hugging Face model ID
matryoshka_dimensions = [384, 256, 128, 64]  # Important: large to small

# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

# load test dataset
test_dataset = load_dataset("json", data_files=test_dataset_path, split="train")
train_dataset = load_dataset("json", data_files=train_dataset_path, split="train")
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
)  # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
)  # Our queries (qid => question)

# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]


matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

In [ ]:
# Evaluate the model
results = evaluator(model)

# # COMMENT IN for full results
# print(results)

# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print
    print(f"{key}: {results[key]}")

# Define loss function

In [ ]:
from sentence_transformers import SentenceTransformerModelCardData, SentenceTransformer

model_id = "sentence-transformers/all-MiniLM-L6-v2"

# load model with SDPA for using Flash Attention 2
model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="BGE base Financial Matryoshka",
    ),
)

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [384, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

# Finetune a model

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

# load train dataset again
train_dataset = load_dataset("json", data_files=train_dataset_path, split="train")

# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="bge-base-financial-matryoshka",  # output directory and hugging face model ID
    num_train_epochs=4,  # number of epochs
    per_device_train_batch_size=32,  # train batch size
    gradient_accumulation_steps=16,  # for a global batch size of 512
    per_device_eval_batch_size=16,  # evaluation batch size
    warmup_ratio=0.1,  # warmup ratio
    learning_rate=2e-5,  # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",  # use constant learning rate scheduler
    optim="adamw_torch_fused",  # use fused adamw optimizer
    tf32=True,  # use tf32 precision
    bf16=True,  # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",  # evaluate after each epoch
    save_strategy="epoch",  # save after each epoch
    logging_steps=10,  # log every 10 steps
    save_total_limit=3,  # save only the last 3 models
    load_best_model_at_end=True,  # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
)

In [ ]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,  # training arguments
    train_dataset=train_dataset.select_columns(
        ["positive", "anchor"]
    ),  # training dataset
    loss=train_loss,
    evaluator=evaluator,
)

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save the best model
trainer.save_model()

# push model to hub
# trainer.model.push_to_hub("bge-base-financial-matryoshka")

# Evaluate fine-tuned model against baseline

In [ ]:
from sentence_transformers import SentenceTransformer

fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluate the model
results = evaluator(fine_tuned_model)

# # COMMENT IN for full results
# print(results)

# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")